# Concurent programming

Sequential Programming: is a form of computing that executes the same sequence of instructions and always produces the same result
- execution is deterministic (assuming no deliberate use of randomness)
- Textual order of the source code doesn't define the order of execution

Concurrent Programming: is a form of computing where threds cn run simultaneously

Different executions of a concurent program may produce different instruction offerings:
- Texture order of source code define the order of execution

```
new Thread(() ->
           computationA()).
           Start();

new Thread(() -> 
           computationB()).
           Start();
           
new Thread(() ->
           computationC()).
           start();
```
computationA(), computationB(), and computationC() can run in any order after their threads start executing.
- Operations are permitted to overlap in time across multiple cores
- Concurrent programming is often used to offlead work form user(UI) thread to background thread(s)
- - Background thread(s) can block
- - UI threads does not block
- - Any mutable state shared between these threads must be protectted to avoid concurrency hazards
- - - Motivates the need for various types of synchronizers

## Concurrency in Java
Concurrency is commmonly used to offload work from the user interface(UI) threads to background threads()

Java Thread is an object 
- it contains methods and field

|Class Thread|
|-|
|java.lang Object java.lang.Thread|
|All Implemented Interface: Runnable|
|Direct Known Subclasses: ForkJoinWorkerThread|
|public class Thread extends Object implements Runnable|
|A thread is a thread of execution in a program. lThe Java Virtual Machine allows an application oto have multiple threads of execution running concurrently|
|Every thread has a priority. Threads with higher priority are executed in preference to threads with lower priority. Each thread may or may not also be marked as a daemon. When code running in some thread creates a new Thread object, the new thread has its prtiority initially set equal to the priority of the creating thread, and is a daemon thread if and only if the creating thread is a daemon.

Each Java thread has its own unique stack, registers, thread-specific storage, etc.

<img src="media/threadstack.png"/>

A Java thread can be in one of the various states
- a Java thread starts out in `New` state

<img src="media/threadstate.png"/>

Concurrent Java threads interact via shared objects and/or messages

### Shared objects
- Synchronized concurrent operations on objects to ensure certain properties
- - Mutual Exclusion
- - - Interation between treads does not corrupt shared mutable data
- - Coordination
- - - Operations occur in the right order at the right time and under the right conditions
<img src="media/sharedobject.png"/>

Example of Java synchronizers:
- Synchronized statements/methods
- Reentrant locks and intrinsic locks
- Atomic operations
- Semaphores
- Condition objects
- "Compare-and-swap" CAS operation in sun.misc.unsafe

### Message passing
- Send messages(s) from producer thread(s) to consumer thread(s) via a threaed-safe queue
<img src="media/messagepassing.png"/>

Examples of Java thread-safe quueues
- Array and linked blocking queues
- Priority blocking queue
- Synchronous queue
- Concurrent linked queue


Java shard objects and message passing are designed to share resources safely and avoid concurrency hazards
- Race conditions: race conditions occur when a program depend upon the sequence or timing of threads for it to operate properly
- Memory inconsistencies: errors that occur when different threads have inconsistent views of what should be the same data
- Deadlocks: Occurs when 2+ competing threads are waiting for the others to finish, and huse never ever do.

# Introduction to Threads
Concurrent apps use threads to simultaneously run multiple computations that potentially interact with each other
- Threads are the most basic way of obtaining concurrency in Java
- A Java thread is a unit of computation that runs in the context of a process
- Process: A process is a unit of resource allocation and protection in Java
- A Java thread runs on one or more cores during its lifetime
- Java enables multiple threads to run in multiple processes atop multiple cores
<img src="media/processthread.png"/>

- Java threads running in the same process can communicate with each other via shard objects or message passing

- Java threads in different processes communicate via shard memory or inter-process communication (IPC) mechanism
<img src="media/interprocesscommunication.png"/>


Each Java thread leverage unique "state" from the underylying OS thread, e.g., a runtime stack, an instruction counter and other registers.
<img src="media/underlyingosthread.png"/>


## Ways of Giving Code to Java Threads
Java threads must be given code to run
- Do not use the "no argument" Thread constructor directly!!!!
```
Thread t = new Thread();
t.start();
```

There are 2 ways to give code to Java Threads
1. Extend the Thread class
```
public class GCDThread extends Thread {
    public void run() {
        // code to run goes here
    }
}
```
Create and start a thread using a named subclass of Thread
```
Thread gCDThread = new GCDThread();
gCDThread.start();
```
2. Implement the Runnable interface
- Implement the run() hook method of an interface to define the thread's computations
```
public class GCDRunnable implements Runnable {
    public void run() {
        // code to run goes here
    }
}
Runnable gCDRunnable = new GCDRunnable();
new Thread(gcdRunnable).start();
```
Pass the runnable to a new thread object and start it.


or we can Create and start a thead by using an anonymous inner class as the runnable
```
new Thread(new Runnable() {
    public void run(){
        // code to run goes here
    }
}).start();
```
3. Use Java 8 lambda expression (variant of using Runnable interface)
- A lambda expression is an unnamed black of code (with optional parameters) that can be passed around and execute later
```
new Thread(() -> {
    // code to run goes here
}).start();
```
- This approach is unwieldy if the cod to run is long, complex, or needs to be used multiple times
- Another approach is to store the runnable in a variable and pass it to the Thread constructor
```
Runnable r = () -> {
    // code to run goes here
};
new Thread(r).start();
```

## Passing Parameters to a Java Thread
- The run() methods defined in Java Thread and Runnable take no parameters
```
<<Java Class>>
(C) Thread
run(): void
```
-----------
```
<<Java Interface>>
(I) Runnable
run():void
```

### Parameters passed to run() can be supplied via one of two other means, e.g.
1. As parameters to a class constructor
- - By passing the parameter(s) when the runnable or thread is created

In [79]:
public class MainActivity {
    public void runnable() {
        System.out.println("MainThread:" + Thread.currentThread());
        new Thread(new GCDRunnable(this)).start();
    }
    
    public void println(String s) {        
        System.out.println(s);
    }
    
}

In [80]:
public class GCDRunnable implements Runnable {
    private final MainActivity mActivity;
    
    public GCDRunnable(MainActivity mainActivity) {
        mActivity = mainActivity;
    }
    
    public void run() {
        final String threadString = "Spin off thread: " + Thread.currentThread().toString(); 
        mActivity.println(threadString);
    }
}

In [81]:
MainActivity main = new MainActivity();
main.runnable();

2. As parameters to `setter` methods

In [148]:
public class MainActivity {
    public void runnable() {
        System.out.println("MainThread:" + Thread.currentThread());
        Thread gCDThread = new GCDThread().setActivity(this);
        gCDThread.start();
    }
    
    public void println(String s) {        
        System.out.println(s);
    }
}

In [149]:
public class GCDThread extends Thread {
    private MainActivity mActivity;
    
    public GCDThread setActivity(MainActivity activity) {
        this.mActivity = activity;
        return this;
    }
    
    public void run() {
        final String threadString = "Spin off thread: " + Thread.currentThread().toString(); 
        mActivity.println(threadString);
    }
}

In [150]:
MainActivity main = new MainActivity();

In [151]:
main.runnable();

MainThread:Thread[IJava-executor-6,5,main]
Spin off thread: Thread[Thread-33,5,main]

Use Fluent interface to pass parameter(s) when the runnable or thread is created.

# How Java Threads Run
There are multiple layers involved in creating and starting a thread

|Treading and Synchronization Packages|
|-|
|Java Execution Environment (e.g., JVM, ART, etc)|
|System Libraries|
|Operating System Kernal|
|Cores|

- Creating a new thread object doens't allocate a run-time call stack of activation records
- The runtime stack and other thread resources are only allocated after the start() method is called
- The Java execution environment calls a thresd's run() hood method after start() creates its resources
- Each thread can run concurrently and block independently
- Any code can generally run in a thread
- A thread can live as long is its run() hook method hasn't returned
- - The underlying thread scheduler can suspend and resume a thread many times during its lifecycle
- - - Scheduler operations are largely invisible to the user code, as long as synchronization is performed properly
- For a thread to execute `forever`, its run() hook method needs an infinite loop
```
public void run() {
    while(true) { ... }
}
```
- The tread is dead after run() returns 
- - A thread can end normally
- - Or an uncought exception can be thrown
- The join() method allows one thread to waif for another thread to complete, which is a simple form of `barrier synchronization`
- - Or a thread can simply evaporate!
- - The Java execution environment recycles thread resources
- - - e.g., runtime stack of activation records, thread-specific storage, etc.
<img src="media/runningthreads.png" />

### Key Methods in Java Thread Class
Certain Java Thread class methods are used in many concurrent Java programs, e.g.,
- `void setDaemon()`
- - Marks thread as `daemon`
- `void start()`
- - Allocates thread resources and initiates thread execution by calling the run() hook method
- - the start() method can only be called once per thread object
- `void run()`
- - Hook method where user code is supplied
- `void join()`
- - Waits for a thread to finish
- `void sleep(long time)`
- - Sleeps for given time in ms
- `Thread currentThread()`
- - Object for current Thread
- `void interrupt()`
- - Post an interrupt request to a Thread
- `boolean isInterrupted()`
- - Tests whether a thread has been interrupted
- `boolean interrupted()`
- - Test whether current thread has been interrupted 
- `void setPriority(int newPriority)` and `int getPriority()`
- - Set and get the prioprity of a Thread